# Patient Demographic dataframe
## Coding below aims to get the reference code of all pateints recruited in this study
- from January 1, 2009, to December 31, 2023
- male and female 
- age > 18
- estimated glomerular filtration rate (eGFR) of less than 60ml/min/1.73m2 according to the 2021 Chronic Kidney Disease Epidemiology Collaboration (CKD-EPI) formula

## Original csv:
- Reference key: unique for each patient
- Clinic code: unique for each specialy clinic and admission, here only have MG (General Medicine) and GER (Geriatrics), codes related to admission, GOPD and non internal medicine related specialties have been deleted 
- Gender: M or F
- Age: round to year
- Date: Date of Creatinine investigation, format: YYYY-MM-DD
- Creatinine: Round to integer

## Import libraries

In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import glob
import gc

## Retrieve the list of patients from serum creatinine investigation 2009 - 2023

In [2]:
path = '/mnt/d/pydatascience/g3_regress/data/Cr'
file_pattern = f"{path}/RRT*.csv"
dataframes = [pd.read_csv(filename) for filename in glob.glob(file_pattern)]
combined_df = pd.concat(dataframes, ignore_index=True)
# Drop rows where 'LIS Result (28 days) - LIS Result: Numeric Result' is missing
combined_df = combined_df.dropna(subset=['LIS Result (28 days) - LIS Result: Numeric Result'])
# Drop rows where both 'Sex' and 'Reference Key' are missing
combined_df = combined_df.dropna(subset=['Sex', 'Reference Key'], how='all')
# Create a DataFrame with unique 'Reference Key' and non-null 'Sex'
reference_df = combined_df.dropna(subset=['Sex']).drop_duplicates(subset=['Reference Key'])
reference_df = reference_df[['Reference Key', 'Sex']]
# Merge the original DataFrame with this reference DataFrame on 'Reference Key' to fill missing 'Sex'
# This uses a left join to ensure all original rows in combined_df are preserved
combined_df = combined_df.merge(reference_df, on='Reference Key', suffixes=('', '_filled'), how='left')
# Fill missing 'Sex' values using the 'Sex_filled' values
combined_df['Sex'] = combined_df['Sex'].fillna(combined_df['Sex_filled'])
# Now that 'Sex' is filled, drop the auxiliary 'Sex_filled' column
combined_df.drop(columns='Sex_filled', inplace=True)
# Drop rows where both 'Sex' and 'Date of Birth (yyyy-mm-dd)' are missing
combined_df = combined_df.dropna(subset=['Sex', 'Date of Birth (yyyy-mm-dd)'])
combined_df.info()
del [reference_df, dataframes, file_pattern, path]
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Index: 2781568 entries, 0 to 2784967
Data columns (total 7 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   Reference Key                                      float64
 1   Date of Birth (yyyy-mm-dd)                         object 
 2   Sex                                                object 
 3   LIS Reference Datetime                             object 
 4   LIS Case No.                                       object 
 5   LIS Result (28 days) - LIS Result: Numeric Result  float64
 6                                                      object 
dtypes: float64(2), object(5)
memory usage: 169.8+ MB


0

## Retrieve patient demographics dataframe
- key: unique for each patient
- dob: Date of birth
- gender: M or F

In [3]:
# Isolate each unique Reference Key with the corresponding Date of Birth, and Gender
demographic_df = combined_df.drop_duplicates(subset=['Reference Key','Sex', 'Date of Birth (yyyy-mm-dd)'])
demographic_df = demographic_df[['Reference Key', 'Date of Birth (yyyy-mm-dd)', 'Sex']].rename(
    columns={'Reference Key': 'key', 'Date of Birth (yyyy-mm-dd)': 'dob', 'Sex': 'gender'}
)
demographic_df['dob'] = pd.to_datetime(demographic_df['dob'], format='mixed')
demographic_df = demographic_df.drop_duplicates()

## Retrieve Creatinine dataframe
- key: unique for each patient
- date: date and time of investigation
- code: clinic or in-patient code of investigation
- Cr: numerical level of serum creatinine

In [4]:
cr_df = combined_df[['Reference Key', 'LIS Reference Datetime', 'LIS Case No.', 'LIS Result (28 days) - LIS Result: Numeric Result']].rename(
    columns={'Reference Key': 'key',
             'LIS Reference Datetime': 'date',
             'LIS Case No.': 'code', 
             'LIS Result (28 days) - LIS Result: Numeric Result': 'Cr'}
)
cr_df['date'] = pd.to_datetime(cr_df['date'], format='mixed')
del combined_df
gc.collect()

31

## Calculate eGFR according to 2021 race free CKD-EPI equation
- Inker Lesley A., Eneanya Nwamaka D., Coresh Josef, Tighiouart Hocine, Wang Dan, Sang Yingying, et al. New Creatinine- and Cystatin C–Based Equations to Estimate GFR without Race. New England Journal of Medicine. 2021;385(19): 1737–1749. https://doi.org/10.1056/NEJMoa2102953.


In [5]:
# Merge cr_df with demographic_df on the 'key' column
egfr_df = pd.merge(cr_df, demographic_df, on='key', how ='outer')
egfr_df = egfr_df.drop_duplicates()
# Convert creatinine from µmol/L to mg/dL
egfr_df['Cr_mg_dL'] = egfr_df['Cr'] / 88.4

# Calculate age in years at the time of the creatinine measurement
egfr_df['age'] = (egfr_df['date'] - egfr_df['dob']).dt.days / 365.25

# Constants based on sex
conditions = [
    egfr_df['gender'] == 'F',
    egfr_df['gender'] == 'M'
]

choices_k = [0.7, 0.9]  # kappa values for female and male
choices_alpha = [-0.241, -0.302]  # alpha values for female and male
egfr_df['kappa'] = np.select(conditions, choices_k, default=np.nan)
egfr_df['alpha'] = np.select(conditions, choices_alpha, default=np.nan)

# Calculate eGFR using the CKD-EPI 2021 equation
egfr_df['eGFRcr'] = 142 * (egfr_df['Cr_mg_dL'] / egfr_df['kappa']).clip(upper=1)**egfr_df['alpha'] * \
                        (egfr_df['Cr_mg_dL'] / egfr_df['kappa']).clip(lower=1)**(-1.2) * \
                        0.9938**egfr_df['age']

# Adjust eGFR for females
egfr_df.loc[egfr_df['gender'] == 'F', 'eGFRcr'] *= 1.012
# Clean the 'code' column by removing '?'
egfr_df['code'] = egfr_df['code'].str.replace('?', '', regex=False)
# Sort the DataFrame by 'key', 'code', and 'date' to prepare for filtering
egfr_df_sorted = egfr_df.sort_values(by=['key', 'code', 'date'])
# Create a flag to identify 'RRT' codes, convert to boolean and fill NaN
egfr_df_sorted['is_rrt'] = egfr_df_sorted['code'].str.startswith('RRT').astype(bool).fillna(False)
# Create a flag to identify 'HN' codes, convert to boolean and fill NaN
egfr_df_sorted['is_hn'] = egfr_df_sorted['code'].str.startswith('HN').astype(bool).fillna(False)
# Mark the last row in each group of 'key' and 'code'
egfr_df_sorted['last_in_group'] = egfr_df_sorted['date'] == egfr_df_sorted.groupby(['key', 'code'])['date'].transform('max')
egfr_df_sorted['last_in_group'] = egfr_df_sorted['last_in_group'].astype(bool).fillna(False)
# Filter rows: Exclude rows where the code starts with 'RRT', and for 'HN' codes, keep only the last in the group
egfr_df_filtered = egfr_df_sorted[~(egfr_df_sorted['is_hn'] & ~egfr_df_sorted['last_in_group']) & ~egfr_df_sorted['is_rrt']]
# Drop the columns used for filtering to clean up the DataFrame
egfr_df_final = egfr_df_filtered.drop(['is_hn', 'last_in_group', 'is_rrt'], axis=1)
# Final DataFrame sorted by 'key' and 'date'
egfr_df = egfr_df_final.sort_values(by=['key', 'date'])
# Calculate age in years
egfr_df['age'] = (egfr_df['date'] - egfr_df['dob']).dt.days / 365.25
egfr_df['age'] = egfr_df['age'].round().astype(int)
# Filter out rows where 'age' is less than 18
egfr_df = egfr_df[egfr_df['age'] >= 18]
# Count the number of entries for each 'key'
egfr_df['entry_count'] = egfr_df.groupby('key')['key'].transform('count')
# Filter out rows where the 'key' has only one entry
egfr_df = egfr_df[egfr_df['entry_count'] > 1]
# Drop the 'entry_count' column as it's no longer needed after filtering
egfr_df.drop(columns=['entry_count', 'Cr_mg_dL', 'kappa', 'alpha'], inplace=True)

# Display the results
display(egfr_df.head())

del [egfr_df_sorted, egfr_df_final, egfr_df_filtered, choices_alpha, choices_k, conditions]
gc.collect()


,key,date,code,Cr,dob,gender,age,eGFRcr
0,449.0,2009-04-08 11:12:00,HN08019759M,125.0,1930-02-22,M,79,50.479592
2,449.0,2009-06-22 09:56:00,HN09012526S,108.0,1930-02-22,M,79,60.082047
3,449.0,2009-06-30 09:03:00,MG0910744O,120.0,1930-02-22,M,79,52.939103
4,449.0,2009-07-07 15:03:00,HN09013776M,116.0,1930-02-22,M,79,55.130600
5,449.0,2009-07-08 11:04:00,HN090656875,129.0,1930-02-22,M,79,48.531896


13

## Retrieve list of reference key, date of first diagnosed CKD with eGFR less than starting point (now we use 60) and date of first diagnosed with eGFR less than end point (now we use 10)

In [6]:
def find_persistent_low_egfr(df, threshold, days):
    # Ensure the DataFrame is sorted by 'key' and 'date'
    df = df.sort_values(by=['key', 'date'])
    column_name = f'first_sub_{threshold}_date'
    # Initialize a column to store the valid end date when conditions are met
    df[column_name] = pd.NaT

    # Iterate over each unique key
    for key, group in df.groupby('key'):
        i = 0
        while i < len(group):
            row = group.iloc[i]
            if row['eGFRcr'] < threshold:
                # Start the check from this row
                j = i
                valid_end_date = None
                # Continue to find a valid range
                while j < len(group):
                    # Check if the day difference meets the requirement
                    if (group.iloc[j]['date'] - row['date']).days > days:
                        # Calculate mean eGFRcr over this range
                        mean_egfr = group.iloc[i:j+1]['eGFRcr'].mean()
                        if mean_egfr < threshold:
                            valid_end_date = group.iloc[j]['date']
                            break  # End date found, break the inner loop
                        else:
                            break  # Mean eGFR not valid, break the inner loop
                    j += 1
                if valid_end_date:
                    df.loc[(df['key'] == key) & (df['date'] == valid_end_date), column_name] = valid_end_date
                    break  # Move to the next key since a valid period is found
            i += 1

    return df

sub_60_df = find_persistent_low_egfr(egfr_df, threshold=60, days=90)
sub_10_df = find_persistent_low_egfr(egfr_df, threshold=10, days=90)

In [7]:
pt_ls = sub_60_df[['key', 'first_sub_60_date']].dropna().reset_index().drop_duplicates()
pt_ls_10 = sub_10_df[['key', 'first_sub_10_date']].dropna().reset_index().drop_duplicates()
pt_ls = pt_ls.merge(pt_ls_10, on='key', how='left').drop(columns=['index_x', 'index_y']).drop_duplicates()
crash_lander_ls = pt_ls[(pt_ls['first_sub_60_date'] >= pt_ls['first_sub_10_date'])]
print('Total number of patients:', pt_ls['key'].nunique())
print('Excluded number of patients who present with eGFR < 10ml/min/1.73m2 (crash landers):', crash_lander_ls['key'].nunique())
pt_ls = pt_ls[~pt_ls['key'].isin(crash_lander_ls['key'].unique())]
print('Number of patients after excluding crash lander:', pt_ls['key'].nunique())
print('Total number of patients developed eGFRcr < 10ml/min/1.73m2:', pt_ls['first_sub_10_date'].notnull().sum())
pt_ls.to_csv('/mnt/d/pydatascience/g3_regress/data/pt_ls.csv', index=False)
min_dates = egfr_df.groupby('key')['date'].min()
keys_after_2019 = min_dates[min_dates > '2019-01-01'].index
test_ls = pt_ls[pt_ls['key'].isin(keys_after_2019)]
test_ls.to_csv('/mnt/d/pydatascience/g3_regress/data/test_ls.csv', index=False)
print('Total number of new patients after 2019-01-01 as test set:', test_ls['key'].nunique())
print('Total number of patients developed eGFRcr < 10ml/min/1.73m2 in test set:', test_ls['first_sub_10_date'].notnull().sum())

del [pt_ls_10]
gc.collect()

Total number of patients: 39048
Excluded number of patients who present with eGFR < 10ml/min/1.73m2 (crash landers): 1132
Number of patients after excluding crash lander: 37916
Total number of patients developed eGFRcr < 10ml/min/1.73m2: 3614
Total number of new patients after 2019-01-01 as test set: 6066
Total number of patients developed eGFRcr < 10ml/min/1.73m2 in test set: 239


0

In [11]:
demographic_df.to_csv('/mnt/d/pydatascience/g3_regress/data/demographic_df.csv', index=False)
gc.collect()

0

In [8]:
def format_and_impute_dates(date_series):
    default_date = datetime.strptime('31/12/2023 23:59:59', '%d/%m/%Y %H:%M:%S')
    date_series = date_series.fillna(default_date)

    # Convert to string format "yyyy/mm/dd"
    return date_series.dt.strftime('%Y/%m/%d')

# Apply formatting and imputation
pt_ls_2 = pt_ls.copy()
pt_ls_2['first_sub_60_date'] = format_and_impute_dates(pt_ls_2['first_sub_60_date'])
pt_ls_2['first_sub_10_date'] = format_and_impute_dates(pt_ls_2['first_sub_10_date'])
pt_ls_2['key'] = pt_ls_2['key'].astype(int)


# Function to split DataFrame and save to files
def save_to_files(df, max_rows=500):
    # Calculate number of files needed
    num_files = (len(df) + max_rows - 1) // max_rows
    
    # Loop to create each file
    for i in range(num_files):
        # Slice the DataFrame
        start_idx = i * max_rows
        end_idx = start_idx + max_rows
        df_slice = df.iloc[start_idx:end_idx]
        
        # File path
        file_path = f'/mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_{i+1}.txt'
        
        # Save to text file with tab delimiter
        df_slice.to_csv(file_path, sep='\t', index=False, header=False)
        print(f"Saved to {file_path}")

# Apply the function to save files
save_to_files(pt_ls_2)
gc.collect()

Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_1.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_2.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_3.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_4.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_5.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_6.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_7.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_8.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_9.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_10.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_11.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_12.txt
Saved to /mnt/d/pydatascience/g3_regress/data/pt_ls/fine_output_keys_13.txt
Saved to /mnt/d/pydat

0

In [71]:
def save_keys_in_chunks(df, max_keys=2000):
    # Ensure the DataFrame only contains the 'key' column
    keys_df = df[['key']]

    # Convert 'key' to integers
    keys_df['key'] = keys_df['key'].astype(int)

    # Calculate the number of chunks needed
    num_chunks = (len(keys_df) + max_keys - 1) // max_keys

    for i in range(num_chunks):
        # Slice the DataFrame
        start_idx = i * max_keys
        end_idx = start_idx + max_keys
        chunk = keys_df.iloc[start_idx:end_idx]

        # File path
        file_path = f'keys_chunk_{i+1}.txt'

        # Save to a tab-delimited text file without header
        chunk.to_csv(file_path, sep='\t', index=False, header=False)
        print(f"Saved to {file_path}")

# Assuming pt_ls_2 is already defined
save_keys_in_chunks(pt_ls_2)
gc.collect()

/tmp/ipykernel_15567/2214653394.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keys_df['key'] = keys_df['key'].astype(int)


Saved to keys_chunk_1.txt
Saved to keys_chunk_2.txt
Saved to keys_chunk_3.txt
Saved to keys_chunk_4.txt
Saved to keys_chunk_5.txt
Saved to keys_chunk_6.txt
Saved to keys_chunk_7.txt
Saved to keys_chunk_8.txt
Saved to keys_chunk_9.txt
Saved to keys_chunk_10.txt
Saved to keys_chunk_11.txt
Saved to keys_chunk_12.txt
Saved to keys_chunk_13.txt
Saved to keys_chunk_14.txt
Saved to keys_chunk_15.txt
Saved to keys_chunk_16.txt
Saved to keys_chunk_17.txt
Saved to keys_chunk_18.txt
Saved to keys_chunk_19.txt


31